In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd


In [ ]:
from pathlib import Path

BASE_PATH = Path.cwd()
print("BASE_PATH:", BASE_PATH)


BASE_PATH: C:\Users\USER\git\link_dl\_03_homeworks\homework_2


In [ ]:
import sys
sys.path.append(BASE_PATH)

In [ ]:
from titanic_dataset import get_preprocessed_dataset

# 데이터 로더 함수

#### 데이터셋 불러와 학습·검증·테스트용 DataLoader 생성

In [ ]:
def get_data():
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    print(f"Train: {len(train_dataset)}, Validation: {len(validation_dataset)}, Test: {len(test_dataset)}")

    train_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset), shuffle=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

    return train_loader, validation_loader, test_loader

# 입력 차원과 출력 차원, 활성화함수에 따라 3층 신경망 모델 정의

#### PyTorch nn.Module과 nn.Sequential을 사용한 다층 퍼셉트론 방식으로 구현

In [ ]:
class MyTitanicModel(nn.Module):
    def __init__(self, n_input, n_output, activation_fn):
        super().__init__()

        act_fn = {
            "relu": nn.ReLU(),
            "sigmoid": nn.Sigmoid(),
            "elu": nn.ELU(),
            "leakyrelu": nn.LeakyReLU()
        }[activation_fn.lower()]

        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            act_fn,
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            act_fn,
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output)
        )

    def forward(self, x):
        return self.model(x)


# 모델과 Adam 옵티마이저를 생성해 반환

In [ ]:
def get_model_and_optimizer(n_input):
    model = MyTitanicModel(n_input=n_input, n_output=1, activation_fn=wandb.config.activation)
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
    return model, optimizer


# 모델을 학습하고 검증 손실을 계산하며, 결과를 wandb에 로깅하는 학습 루프

#### 손실 함수: BCEWithLogitsLoss (이진 분류용)

#### 학습: train_loader로 미니배치 학습 → optimizer.zero_grad() → loss.backward() → optimizer.step() 순서

#### 검증: valid_loader로 평가, torch.no_grad() 사용

#### 로그: wandb.log로 epoch별 학습·검증 손실 기록

#### 출력: 일정 epoch마다 학습/검증 손실 출력

In [ ]:
def training_loop(model, optimizer, train_loader, valid_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.BCEWithLogitsLoss()
    next_print_epoch = 50

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            X, y = batch["input"], batch["target"].float().unsqueeze(1)

            optimizer.zero_grad()
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in valid_loader:
                X, y = batch["input"], batch["target"].float().unsqueeze(1)
                y_pred = model(X)
                loss = loss_fn(y_pred, y)
                val_loss += loss.item()

        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss / len(train_loader),
            "val_loss": val_loss / len(valid_loader)
        })

        if epoch % next_print_epoch == 0:
            print(f"Epoch {epoch}: train_loss={train_loss / len(train_loader):.4f}, val_loss={val_loss / len(valid_loader):.4f}")

# 모델로 테스트 데이터를 예측해 이진 레이블 생성 후 CSV 제출 파일로 저장

#### model.eval()과 torch.no_grad()로 테스트 데이터 예측

#### torch.sigmoid로 확률 변환 후 0.5 기준 이진 레이블 결정

#### 결과를 pandas.DataFrame으로 만들어 CSV 파일로 저장

In [ ]:
def generate_submission(model, test_loader, output_path="submission.csv"):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for batch in test_loader:
            X = batch["input"]
            y_pred = model(X)
            y_prob = torch.sigmoid(y_pred)
            y_label = (y_prob >= 0.5).int().squeeze()
            all_preds.extend(y_label.tolist())

    passenger_ids = list(range(892, 892 + len(all_preds)))  # Titanic test set ID 범위

    submission = pd.DataFrame({
        "PassengerId": passenger_ids,
        "Survived": all_preds
    })
    submission.to_csv(output_path, index=False)
    print(f"✅ Submission file saved: {output_path}")

# 데이터 로드, 모델 생성·학습, 테스트 예측 후 wandb로 실험 기록하고 제출 CSV 생성까지 전체 파이프라인 실행

### 파이프라인 기반 순차 실행 방식

#### wandb로 실험 설정/로그 초기화

#### 데이터 로드 → 모델 및 옵티마이저 생성 → 학습 루프 수행

#### 테스트 데이터 예측 → CSV 제출 파일 생성

#### wandb 실험 종료

In [ ]:
def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [32, 16],
        'activation': args.activation
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="titanic_training",
        entity="jjw122601-koreatech",
        notes="Titanic survival prediction",
        tags=["titanic", "binary_classification"],
        name=current_time_str,
        config=config
    )

    train_loader, valid_loader, test_loader = get_data()
    sample_batch = next(iter(train_loader))
    n_input = sample_batch["input"].shape[1]
    print(f"Detected n_input = {n_input}")

    model, optimizer = get_model_and_optimizer(n_input)
    training_loop(model, optimizer, train_loader, valid_loader)

    output_csv = f"submission_{wandb.config.activation}_b{wandb.config.batch_size}.csv"
    generate_submission(model, test_loader, output_path=output_csv)

    wandb.finish()


 ![이미지](https://i.ifh.cc/DOvsP4.png)

## 명령행 인자를 받아 학습 설정을 지정하고, main 함수로 전체 학습·예측 파이프라인 실행

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument("--wandb", action=argparse.BooleanOptionalAction, default=False)
    parser.add_argument("-b", "--batch_size", type=int, default=32)
    parser.add_argument("-e", "--epochs", type=int, default=200)
    parser.add_argument("-a", "--activation", type=str, default="relu",
                        choices=["relu", "sigmoid", "elu", "leakyrelu"],
                        help="Activation function")

    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--wandb | --no-wandb] [-b BATCH_SIZE] [-e EPOCHS] [-a {relu,sigmoid,elu,leakyrelu}]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\USER\AppData\Roaming\jupyter\runtime\kernel-0af2efc6-4138-405e-8afa-76c63b708191.json


SystemExit: 2

# 숙제 후기

### 이번 프로젝트에서는 Titanic 데이터셋을 이용해 생존 여부를 예측하는 3층 다층 퍼셉트론 모델을 구현했습니다. 데이터는 전처리 후 학습, 검증, 테스트용으로 나누고 DataLoader로 배치 단위로 불러왔으며, BCEWithLogitsLoss로 학습하고 검증 손실을 wandb로 기록했습니다. 학습 후 테스트 데이터를 예측해 sigmoid로 이진 레이블을 만들고 CSV 파일로 저장했습니다. 특히 손실이 불안정하게 변동하거나 학습 속도와 배치 크기, 활성화 함수 선택으로 모델 성능 최적화가 어려웠지만, 여러 실험을 반복하며 하이퍼파라미터를 조정하고 wandb 기록을 참고하여 최적의 설정을 찾아 문제를 해결할 수 있었습니다.